## Cabeçalho
### Objetivo
Limpar/Tratar os dados de despesas disponíveis em tb_despesas_bronze extraídos da API de dados abertos da Câmara dos Depoutados.

**Sobre
Data de crição**: 06/10/2025

**Responsável:** Wandrieli Nery Barbosa

Parâmetros

In [0]:
%run /Users/wandrieli.89@gmail.com/projeto_tiller/Parametros/nb_parametros

In [0]:
%run /Users/wandrieli.89@gmail.com/projeto_tiller/Parametros/nb_catalag_schema

Transformação

In [0]:
set_catalog_and_schema_silver() #Define catalog e schema conforme run


def process_despesas_silver():

    try:

        df_bronze = spark.table(
            "tiller.bronze.tb_despesas_bronze"
        )  # Carrega a tabela bronze

        if df_bronze.count() == 0:
            print("DataFrame Bronze vazio. Nenhum registro para processar.")
            return None

        df_silver = (
            df_bronze.withColumn("data_emissao", F.to_date("datEmissao"))
            .withColumn("ano", F.col("numAno").cast("int"))
            .withColumn("mes", F.col("numMes").cast("int"))
            .withColumn("valor_documento", F.col("vlrDocumento").cast("double"))
            .withColumn("valor_glosa", F.col("vlrGlosa").cast("double"))
            .withColumn("valor_liquido", F.col("vlrLiquido").cast("double"))
            .withColumn("data_ingestao_silver", F.current_timestamp())
            
            .withColumn(
                "txtFornecedor",
                F.when(F.col("txtFornecedor").isNull(), "Não informado").otherwise(
                    F.col("txtFornecedor") # Tratar valores nulos
                ),
            )
            .withColumn(
                "txtNumero",
                F.when(F.col("txtNumero").isNull(), "Não informado").otherwise(
                    F.col("txtNumero")
                ),
            )
            .withColumn("txtDescricao", F.initcap(F.trim("txtDescricao")))
        )

        print("Schema do DataFrame Silver:")
        df_silver.printSchema()

        print("Amostra de registros Silver:")
        df_silver.show(5, truncate=False)

        df_silver.write\
            .format("delta")\
            .mode("overwrite")\
            .option("overwriteSchema", "true")\
            .saveAsTable("tb_despesas_silver")  # Salva a tabela Silver

        print("Tabela criada/atualizada com sucesso!")
        return df_silver

    except Exception as e:
        print(f"Erro ao processar despesas Silver: {e}")
        return None


despesas_silver = process_despesas_silver()  # Executa o processamento

In [0]:
%sql
select
  *
from
  tiller.silver.tb_despesas_silver;